In [39]:
#library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as ap

In [40]:
#mount file from google drive
from google.colab import drive
drive.mount('/content/drive')
#grab the data
img224 = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/img224.npy')
label  = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/label.npy')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
X224_train = img224
y224_train = label
y224_train = tf.keras.utils.to_categorical(y224_train, 4)

In [42]:
#checkpoint
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/vgg'
model_checkpoint_callack = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weight_only = True,
    save_freq = 'epoch',
    mode = 'auto',
    save_best_only = True,
    monitor = 'val_accuracy'
)

In [43]:
#VGG16
def model_vgg16(X_train, y_train,epochs = 8):
    base_model = ap.VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
    base_model.trainable = False

    head_model = base_model.output
    head_model = keras.layers.Flatten()(head_model)
    head_model = keras.layers.Dense(512, activation = 'relu')(head_model)
    head_model = keras.layers.Dropout(0.5)(head_model)
    head_model = keras.layers.Dense(128, activation = 'relu')(head_model)
    head_model = keras.layers.Dense(4,activation='softmax')(head_model)

    model = keras.Model(base_model.input,head_model)

    callbacks = [keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/FINAL_DL/logs/vgg'),model_checkpoint_callack]
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train,batch_size=32, epochs=epochs, callbacks = callbacks, validation_split = 0.2)

    return model,history

In [44]:
model_vgg16_m1,history= model_vgg16(X224_train,y224_train,32)

Epoch 1/32
75/75 [==============================] - 89s 1s/step - loss: 1.6541 - accuracy: 0.5042 - val_loss: 0.6716 - val_accuracy: 0.7517
Epoch 2/32
75/75 [==============================] - 88s 1s/step - loss: 0.7184 - accuracy: 0.7079 - val_loss: 0.5300 - val_accuracy: 0.7600
Epoch 3/32
75/75 [==============================] - 88s 1s/step - loss: 0.5690 - accuracy: 0.7663 - val_loss: 0.4753 - val_accuracy: 0.8267
Epoch 4/32
75/75 [==============================] - 85s 1s/step - loss: 0.5524 - accuracy: 0.7667 - val_loss: 0.5689 - val_accuracy: 0.7500
Epoch 5/32
75/75 [==============================] - 88s 1s/step - loss: 0.5047 - accuracy: 0.8029 - val_loss: 0.3910 - val_accuracy: 0.8600
Epoch 6/32
75/75 [==============================] - 85s 1s/step - loss: 0.5055 - accuracy: 0.8004 - val_loss: 0.4160 - val_accuracy: 0.8383
Epoch 7/32
75/75 [==============================] - 84s 1s/step - loss: 0.4300 - accuracy: 0.8275 - val_loss: 0.3762 - val_accuracy: 0.8600
Epoch 8/32
75/75 [==

In [47]:
model_vgg16_m1.save_weights('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/weights/vgg_m1_weights.h5')

In [48]:
#save the curve
his = pd.DataFrame(history.history)
his.to_pickle('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/curve/history_vgg.pkl')

In [52]:
model_vgg16_m1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   